<a href="https://colab.research.google.com/github/Prakruthi-G26/DevOps/blob/main/HealthAgentAI.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

#Health Agent
An health agent to ensure healthy lives. It performs following:

* Analyse patient data and extract information like age, symptoms, diet and exercise from patient data
* Predict chronic disease risks (Diabetes, Hypertension (BP), cancer (basic risk))
* Personalised recommondations based on identified risks and lifestyle factors



# STEP 0: Install dependencies

In [1]:
!pip install -qU google-generativeai==0.8.5 google-ai-generativelanguage==0.6.15 \
langgraph langchain langchain-google-genai openai

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 43.7/43.7 kB 3.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 152.5/152.5 kB 5.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 42.0/42.0 kB 2.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 767.8/767.8 kB 26.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 43.9/43.9 kB 2.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 50.6/50.6 kB 2.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 216.5/216.5 kB 12.9 MB/s eta 0:00:00


# STEP 1: Imports and secure API key input

In [2]:
import os
import getpass
from langgraph.graph import StateGraph, END
from langchain_google_genai import ChatGoogleGenerativeAI
from langchain_core.messages import HumanMessage

# secure gemini API key

In [3]:
os.environ["GOOGLE_API_KEY"] = getpass.getpass("ENTER YOUR API Key:")

ENTER YOUR API Key:··········


# STEP 2: Initialize Gemini 1.5 Flash

In [4]:
llm = ChatGoogleGenerativeAI(model="models/gemini-1.5-flash-latest", temperature=0.2)

# WORKFLOW

1. Getting patient data

In [5]:
def get_symptom(state: dict)->dict:
  user_input = input("welcome. Please enter your age, symptom and lifestyle(like if you do any excercises or diet): ")
  state["user_input"] = user_input
  return state

2. Extracting patient information

In [17]:
from langchain_core.messages import HumanMessage

def extract_health_info(state: dict) -> dict:
    prompt = (
        "You're a medical assistant. Extract the following health info from the text below:\n"
        "- Age (as number)\n"
        "- Symptoms (as list of words)\n"
        "- Diet (healthy/unhealthy)\n"
        "- Exercise (regular/none)\n"
        "Respond ONLY in JSON format with keys: age, symptoms, diet, exercise.\n\n"
        f"User input: {state['user_input']}"
    )

    response = llm.invoke([HumanMessage(content=prompt)])
    print("Extracted Info:", response.content)  # Debug
    extracted=response.content.strip()
    state["patient_info"] = extracted
    #
    # try:
    #     extracted = eval(response.content)  # Safer to use json.loads in production
    #     state["patient_info"] = extracted
    # except Exception as e:
    #     print("Failed to parse LLM response:", e)
    #     state["patient_info"] = {}

    return state


3. Predicting chronic disease risks

In [20]:
def predict_disease_risk(state: dict) -> dict:
    info = state['patient_info']
    prompt = (
        "You are a medical AI. Based on the patient info below, predict the likelihood (High/Medium/Low)"
        " of developing these chronic diseases:\n"
        "- Diabetes\n"
        "- Blood Pressure\n"
        "- Cancer\n\n"
        "Patient Info:\n"
        f"{info}\n\n"
        "Respond in JSON like:\n"
        '{"diabetes_risk": "High", "bp_risk": "Medium", "cancer_risk": "Low"}'
    )

    response = llm.invoke([HumanMessage(content=prompt)])
    print("Risk Prediction:", response.content)  # Debug
    state["risk_prediction"] = response.content.strip()
    # try:
    #     risk = eval(response.content)  # Or use json.loads
    #     state["risk_prediction"] = risk
    # except Exception as e:
    #     print("Failed to parse risk response:", e)
    #     state["risk_prediction"] = {}

    return state


4. Personalised recommendations

In [13]:
def generate_recommendations(state: dict) -> dict:
    info = state.get("patient_info", {})
    risks = state.get("risk_prediction", {})

    prompt = (
        "You're a wellness assistant. Based on the following patient data and disease risk levels, "
        "give personalized, practical, non-technical health recommendations.\n"
        "Respond in bullet points.\n\n"
        f"Patient Info: {info}\n"
        f"Risk Levels: {risks}\n"
    )

    response = llm.invoke([HumanMessage(content=prompt)])
    state["recommendations"] = response.content.strip()
    print("Recommendations:\n", response.content)
    return state


# LangGraph

In [21]:
# Define nodes
graph_builder = StateGraph(dict)
graph_builder.set_entry_point("get_symptom")
graph_builder.add_node("get_symptom", get_symptom)
graph_builder.add_node("Extract", extract_health_info)
graph_builder.add_node("Predict", predict_disease_risk)
graph_builder.add_node("Recommend", generate_recommendations)
# Transitions
graph_builder.add_edge("get_symptom", "Extract")
graph_builder.add_edge("Extract", "Predict")
graph_builder.add_edge("Predict", "Recommend")
graph_builder.add_edge("Recommend", END)
# Compile graph
graph = graph_builder.compile()

In [22]:
result = graph.invoke({})

print("==== Final Output ====\n")
print("Extracted Info:", result['patient_info'])
print("Risk Prediction:", result['risk_prediction'])
print("Recommendations:", result['recommendations'])

welcome. Please enter your age, symptom and lifestyle(like if you do any excercises or diet): I'm 50 years old. I often feel tired, eat a lot of sweets, and I don't exercise at all.
Extracted Info: ```json
{
  "age": 50,
  "symptoms": ["tired"],
  "diet": "unhealthy",
  "exercise": "none"
}
```
Risk Prediction: ```json
{"diabetes_risk": "Medium", "bp_risk": "Medium", "cancer_risk": "Medium"}
```
Recommendations:
 * **Boost your energy levels:** Since you're feeling tired, let's focus on improving your sleep hygiene. Aim for 7-8 hours of quality sleep each night.  Establish a regular sleep schedule, create a relaxing bedtime routine, and make sure your bedroom is dark, quiet, and cool.

* **Revamp your diet:**  An unhealthy diet significantly impacts energy levels and increases your risk for diabetes, high blood pressure, and cancer. Start small!  Swap sugary drinks for water, add more fruits and vegetables to your meals, and choose lean protein sources. Consider consulting a registered